In [2]:
%pip install --upgrade transformers tokenizers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 179.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 144.2 MB/s  0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers━━━━━━━━━━━━━━━━━━ 0/2 [tokenizers]
    Found existing installation: transformers 4.40.20m 0/2 [tokenizers]
    Uninstalling transformers-4.40.2:━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
      Successfully uninstalled transformers-4.40.27m╺━━━━━━━━━━━━━━━━━━━ 1/2 [transformers]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [transformers]0m 1/2 [transformers]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
vllm 0.12.0 requires tiktoken>=0.6.0, but you have tiktoken 0.5.2 which is incompatible.
Note: you ma

In [ ]:
import os
import sys
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Files to save the translations
input_file = "shopping_sentences.txt"
output_nl = "shopping_dutch.txt"
output_de = "shopping_german.txt"

# Load the model
model_id = "Unbabel/Tower-Plus-2B"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

# Load data
with open("shopping_sentences.txt", "r", encoding="utf-8") as f:
    english_lines = [line.strip() for line in f if line.strip()]

# Translation loops
dutch_translations = []
german_translations = []

for idx, line in enumerate(english_lines):
    # Translation to Dutch
    messages_nl = [
        {"role": "user", "content": f"Translate the following English source text to Dutch:\nEnglish: {line}\nDutch: "}
    ]

    # Translation to German
    messages_de = [
        {"role": "user", "content": f"Translate the following English source text to German:\nEnglish: {line}\nGerman: "}
    ]

    for lang, msgs, target_list in [("Dutch", messages_nl, dutch_translations), ("German", messages_de, german_translations)]:
        input_ids = tokenizer.apply_chat_template(
            msgs, 
            add_generation_prompt=True, 
            return_tensors="pt"
        ).to(model.device)
        
        outputs = model.generate(input_ids, max_new_tokens=100, do_sample=False)
        
        # Only save the translation part
        translation = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
        target_list.append(translation.strip())

# Save translations to files
with open(output_nl, "w", encoding="utf-8") as f:
    f.write("\n".join(dutch_translations) + "\n")

with open(output_de, "w", encoding="utf-8") as f:
    f.write("\n".join(german_translations) + "\n")

print(f"Saved translations:")
print(f"Dutch: {output_nl}")
print(f"German: {output_de}")

/home/mbenkaddour/.conda/envs/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
The following generation flags are not valid and may be ignored: ['cache_implementation']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Loading checkpoint shards: 100%|██████████| 2/2 [00:06<00:00,  3.11s/it]


Saved translations:
Dutch: test/shopping_dutch.txt
German: test/shopping_german.txt


In [ ]:
import os
import sys
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

# Files to save the translations
input_file = "shopping_sentences.txt"
output_ar = "shopping_arabic.txt"

# Load the model
model_id = "google/gemma-3-12b-it" 
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id, 
    torch_dtype=torch.bfloat16, 
    device_map="auto"
)

# Load the data
if not os.path.exists(input_file):
    raise FileNotFoundError(f"Bestand {input_file} niet gevonden!")

with open(input_file, "r", encoding="utf-8") as f:
    english_lines = [line.strip() for line in f if line.strip()]

print(f"Start met het vertalen van {len(english_lines)} zinnen naar het Arabisch...")

# Translation loop
arabic_translations = []

for idx, line in enumerate(english_lines):
    messages = [
        {"role": "user", "content": f"Translate the following English sentence to Arabic. Only provide the translation, no extra text.\nEnglish: {line}"}
    ]
    
    input_ids = tokenizer.apply_chat_template(
        messages, 
        add_generation_prompt=True, 
        return_tensors="pt"
    ).to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids, 
            max_new_tokens=150, 
            do_sample=False 
        )
    
    # Just save the translation part
    translation = tokenizer.decode(outputs[0][len(input_ids[0]):], skip_special_tokens=True)
    arabic_translations.append(translation.strip())

# Save the results
with open(output_ar, "w", encoding="utf-8") as f:
    f.write("\n".join(arabic_translations) + "\n")

print(f"Arabic translations saved to: {output_ar}")

Loading checkpoint shards: 100%|██████████| 5/5 [00:29<00:00,  5.81s/it]
The following generation flags are not valid and may be ignored: ['top_p', 'top_k']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Start met het vertalen van 910 zinnen naar het Arabisch...
Arabic translations saved to: test/shopping_arabic.txt
